In [1]:
import os
import pathlib
import pickle
from shutil import unpack_archive
import shutil

import numpy as np
import scipy.stats as stats
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
class Loader:
    def __init__(self, occ, noise, ind, img_num):
        self.noise = noise
        self.img_num = img_num
        
        def gen_path(occ, noise, ind, img_num):
            P_path = os.path.join("PR", occ, noise, ind, str(img_num)+"_pre.pickle")
            R_path = os.path.join("PR", occ, noise, ind, str(img_num)+"_rec.pickle")
            occ_path = os.path.join("preprocessing", "labels", occ+"_l", noise, ind, ind+"_"+str(img_num)+"_occlusion.pickle")
            label_path = os.path.join("preprocessing", "labels", occ+"_l", noise, ind, ind+"_"+str(img_num)+"_label.pickle")
            return P_path, R_path, occ_path, label_path
        
        P_path, R_path, occ_path, label_path = gen_path(occ, noise, ind, img_num)
        with open(str(P_path), 'rb') as f:
            self.P = np.array(pickle.load(f))
        with open(str(R_path), 'rb') as f:
            self.R = np.array(pickle.load(f))
        with open(str(occ_path), 'rb') as f:
            self.occ = np.array(pickle.load(f))
        with open(str(label_path), 'rb') as f:
            self.label = np.array(pickle.load(f))

    def prop_occ(self):
        label_mode, _ = stats.mode(self.label, axis=0)
        label_mode = np.squeeze(label_mode).astype("int")
        self.occ = np.average(self.occ, axis=0)[label_mode]
        
    def prop_PR(self):
        sup_thes = list(range(self.img_num,-1,-4))[:int((len(list(range(self.img_num,-1,-4)))-1)*7/8)]
        global_true_idx = [np.where((self.P>0.99)[:,i])[0]
                           if np.where((self.P>0.99)[:,i])[0].size != 0 
                           else np.array([-1])
                           for i in range(self.P.shape[1])]
        local_opt_idx = [np.argmax(self.R[:,i][global_true_idx[i]]) for i in range(len(global_true_idx))]
        global_opt_idx = [g_idx[l_idx] for g_idx, l_idx in zip(global_true_idx, local_opt_idx)]
        self.opt_sup_th = [sup_thes[opt_idx] for opt_idx in global_opt_idx]
    


In [3]:
occs = ["q_a", "q_aothin", "q_aothick"]
noises = ["0mm", "1mm", "3mm"]
inds = ["q_1_a", "q_2_a", "q_3_a"]
img_nums = [32]

In [4]:
opt_sup_dict = {}
for noise in noises:
    for img_num in img_nums:
        temp_occ_val_list = []
        temp_opt_sup_th_list = []
        for occ in occs:
            for ind in inds:
                lo = Loader(occ, noise, ind, img_num)
                lo.prop_occ()
                lo.prop_PR()
                opt_sup_dict[(occ, noise, ind, img_num)] = lo.opt_sup_th

In [5]:
def temps_folder_path_unzip(occ, noise, ind, img_num):
    ind_folder = pathlib.Path(os.path.join("temps", occ, noise, ind))
    num_files = ind_folder.glob("*.*")
    temp_folder = ind_folder/"temp"
    if temp_folder.exists():
        shutil.rmtree(str(temp_folder))
    for num_file in num_files:
        if num_file.name.split(".")[0] != str(img_num):
            continue
        if num_file.suffix == ".zip":
            unpack_archive(filename=str(num_file), extract_dir=str(ind_folder), format="zip")
        else:
            unpack_archive(filename=str(num_file), extract_dir=str(ind_folder), format="gztar")
    return temp_folder

def get_tags(temp_folder):
    tags_path = list(temp_folder.glob("*.TDlines"))

    tags = []
    for tag_path in tags_path:
        tag_sp = tag_path.stem.split("_")
        tag_arr = ((int(tag_sp[0]), int(tag_sp[1])), tag_sp[2])
        tags.append(tag_arr)
    return tags            

def support_th(tags, temp_folder, sup_th):
    curve_fragment = []
    for tag in tags:
        if tag[1] == "R":
            continue

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.TDlines".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            lines_list = pickle.load(f)

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.support_dict".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            support_list, support_ac = pickle.load(f)
        #support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
        tag_list = []
        for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
            col_curve_fragment = []
            for frag, sup, sup_ac in zip(col, sup_col, ac_col):
                if sup > sup_th:
                    frag = np.reshape(frag,(-1, 3))
                    frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                    col_curve_fragment.append(frag)
            tag_list.append(col_curve_fragment)
        curve_fragment.append(tag_list)
    return curve_fragment

def curve_fragment_align(curve_fragment):
    cfs = [[] for i in range(46)]
    for cols in curve_fragment:
        for i in range(len(cols)):
            cfs[i] += cols[i]
    temp_cfs = []
    for i, cf_label in enumerate(cfs):
        A = [frag  for frag in cf_label if frag.size!=0]
        temp_cfs.append(A)
    cfs = temp_cfs
    return cfs

In [6]:
opt_sup_dict

{('q_a', '0mm', 'q_1_a', 128): [108, 124, 108, 116, 120, 116, 88, 64],
 ('q_a', '0mm', 'q_2_a', 128): [96, 124, 88, 76, 124, 104, 92, 88],
 ('q_a', '0mm', 'q_3_a', 128): [112, 120, 108, 112, 120, 112, 56, 52],
 ('q_aothin', '0mm', 'q_1_a', 128): [96, 120, 92, 108, 116, 108, 68, 52],
 ('q_aothin', '0mm', 'q_2_a', 128): [88, 124, 80, 68, 112, 96, 80, 76],
 ('q_aothin', '0mm', 'q_3_a', 128): [96, 116, 88, 108, 120, 108, 44, 40],
 ('q_aothick', '0mm', 'q_1_a', 128): [84, 120, 80, 104, 116, 104, 60, 44],
 ('q_aothick', '0mm', 'q_2_a', 128): [84, 124, 80, 60, 112, 96, 68, 64],
 ('q_aothick', '0mm', 'q_3_a', 128): [84, 112, 76, 100, 120, 100, 36, 32],
 ('q_a', '0mm', 'q_1_a', 64): [52, 60, 52, 56, 56, 56, 44, 32],
 ('q_a', '0mm', 'q_2_a', 64): [44, 60, 40, 36, 60, 48, 44, 40],
 ('q_a', '0mm', 'q_3_a', 64): [52, 60, 52, 56, 56, 52, 28, 24],
 ('q_aothin', '0mm', 'q_1_a', 64): [44, 56, 44, 52, 56, 52, 32, 24],
 ('q_aothin', '0mm', 'q_2_a', 64): [44, 60, 36, 32, 52, 44, 36, 36],
 ('q_aothin', '0m

In [13]:
for info in opt_sup_dict:
    occ, noise, ind, img_num = info
    output_file = os.path.join("opt_curve_fragment", occ, noise, ind, str(img_num)+"_cfs.pickle")
    if os.path.exists(output_file):
        continue
    temp_folder = temps_folder_path_unzip(occ, noise, ind, img_num)
    tags = get_tags(temp_folder)
    temp_cfs = []
    for i, osth in enumerate(opt_sup_dict[info]):
        curve_fragment = support_th(tags, temp_folder, osth)
        cfs = curve_fragment_align(curve_fragment)
        temp_cfs.append(cfs[i])
        os.makedirs(os.path.join("opt_curve_fragment", occ, noise, ind), exist_ok=True)
        with open(os.path.join("opt_curve_fragment", occ, noise, ind, str(img_num)+"_cfs.pickle"), 'wb') as f:
            pickle.dump(temp_cfs, f)